<div align="center">

  <a href="https://ultralytics.com/yolo" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/main/yolov8/banner-yolov8.png"></a>

  [中文](https://docs.ultralytics.com/zh/) | [한국어](https://docs.ultralytics.com/ko/) | [日本語](https://docs.ultralytics.com/ja/) | [Русский](https://docs.ultralytics.com/ru/) | [Deutsch](https://docs.ultralytics.com/de/) | [Français](https://docs.ultralytics.com/fr/) | [Español](https://docs.ultralytics.com/es/) | [Português](https://docs.ultralytics.com/pt/) | [Türkçe](https://docs.ultralytics.com/tr/) | [Tiếng Việt](https://docs.ultralytics.com/vi/) | [العربية](https://docs.ultralytics.com/ar/)

  <a href="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yml"><img src="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yml/badge.svg" alt="Ultralytics CI"></a>
  <a href="https://colab.research.google.com/github/ultralytics/notebooks/blob/main/notebooks/how-to-monitor-objects-in-queue-using-queue-management-solution.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  
  <a href="https://ultralytics.com/discord"><img alt="Discord" src="https://img.shields.io/discord/1089800235347353640?logo=discord&logoColor=white&label=Discord&color=blue"></a>
  <a href="https://community.ultralytics.com"><img alt="Ultralytics Forums" src="https://img.shields.io/discourse/users?server=https%3A%2F%2Fcommunity.ultralytics.com&logo=discourse&label=Forums&color=blue"></a>
  <a href="https://reddit.com/r/ultralytics"><img alt="Ultralytics Reddit" src="https://img.shields.io/reddit/subreddit-subscribers/ultralytics?style=flat&logo=reddit&logoColor=white&label=Reddit&color=blue"></a>
  
  Welcome to the queue management using Ultralytics <a href="https://docs.ultralytics.com/solutions/">Solutions</a> 🚀 notebook!
  
  Ultralytics <a href="https://github.com/ultralytics/ultralytics">YOLO11</a> is the latest version of the YOLO (You Only Look Once) AI models developed by <a href="https://ultralytics.com">Ultralytics</a>. We hope that the resources in this notebook will help you get the most out of YOLO11 + <a href="https://docs.ultralytics.com/solutions/">Solutions</a>. Please browse the YOLO11 <a href="https://docs.ultralytics.com/">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/ultralytics">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!</div>

# Queue Management using Ultralytics Solutions

This notebook serves as a starting point for [queue management](https://docs.ultralytics.com/guides/queue-management/) in videos or live streams using the Ultralytics Solutions.

### What is Queue Management?

- Queue management involves accurate identification and monitoring of people or objects forming queues in real-time video streams. Ultralytics Solutions enables efficient queue tracking for scenarios like retail store entrances, transport terminals, and event venues, ensuring smooth crowd flow and improved operational oversight.

### Advantages of Queue Management?

- **Crowd Flow Optimization:** Real-time monitoring of queue lengths helps in adjusting staffing or opening new counters, reducing wait times and enhancing user experience.

- **Operational Efficiency:** Insights from queue patterns can optimize service workflows and scheduling in high-traffic environments such as banks, airports, or restaurants.

- **Improved Safety & Compliance:** Detecting overcrowded or stagnant queues enables proactive interventions to maintain safety and follow social distancing or security protocols.

### Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [ ]:
!pip install ultralytics

In [1]:
import ultralytics
import cv2
from ultralytics.utils.downloads import safe_download
from ultralytics import solutions

ultralytics.checks()

Ultralytics 8.3.173 🚀 Python-3.13.5 torch-2.7.1 CPU (Apple M4 Pro)
Setup complete ✅ (14 CPUs, 48.0 GB RAM, 688.6/926.4 GB disk)


### Read the Video File

You can read the video either from a local file or stream it using an RTSP (Real-Time Streaming Protocol) source, offering flexibility based on your input needs. We'll also initialize the video writer to handle saving the processed output video.

In [2]:
safe_download("https://github.com/ultralytics/notebooks/releases/download/v0.0.0/queue-management-demo.mp4")
cap = cv2.VideoCapture("queue-management-demo.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH,
                                       cv2.CAP_PROP_FRAME_HEIGHT,
                                       cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("queue-management.avi",
                               cv2.VideoWriter_fourcc(*"mp4v"),
                               fps, (w, h))

### Set Region Coordinates

In this step, we define the coordinates of the area to be monitored, enabling precise object tracking and efficient queue management within the video or stream. This ensures effective monitoring of objects across designated zones.

In [3]:
# For rectangle region counting
queue_region = [
   (57, 271),
   (295, 669),
   (879, 521),
   (315, 215),
]

# For polygon region counting
# queue_region = [
#     (20, 400),
#     (1080, 400),
#     (1080, 360),
#     (20, 360),
#     (20, 400)
#     ]

### Initialize the QueueManager Class

Now, let's initialize the `QueueManager` class to detect and monitor objects in every frame of the video.

In [4]:
# Init ObjectCounter
queuemanager = solutions.QueueManager(
    show=True,  # Display the output
    region=queue_region,  # Pass region points
    model="yolo11m.pt",  # model="yolo11n-obb.pt" for object counting using YOLO11 OBB model.
    classes=[0],  # If you want to count specific classes i.e person COCO pretrained model.
    line_width=3,  # Adjust the line width for bounding boxes and text display
)

Ultralytics Solutions: ✅ {'source': None, 'model': 'yolo11m.pt', 'classes': [0], 'show_conf': True, 'show_labels': True, 'region': [(57, 271), (295, 669), (879, 521), (315, 215)], 'colormap': 21, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 3, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': True, 'iou': 0.7, 'conf': 0.25, 'device': None, 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': True, 'data': 'images'}


### Process Video Frames

In this step, we process each frame of the video to detect and analyze objects, enabling real-time tracking and queue management based on the visual data within the frames.

In [ ]:
# Process video
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    results = queuemanager(im0)  # queue management
    video_writer.write(results.plot_im)   # write the video frames

cap.release()   # Release the capture
video_writer.release()


0: 674x1280 32.4ms
Speed: 659.4ms track, 32.4ms solution per image at shape (1, 3, 674, 1280)

1: 674x1280 22.2ms
Speed: 73.1ms track, 22.2ms solution per image at shape (1, 3, 674, 1280)

2: 674x1280 16.9ms
Speed: 68.0ms track, 16.9ms solution per image at shape (1, 3, 674, 1280)

3: 674x1280 15.2ms
Speed: 66.9ms track, 15.2ms solution per image at shape (1, 3, 674, 1280)

4: 674x1280 15.2ms
Speed: 71.3ms track, 15.2ms solution per image at shape (1, 3, 674, 1280)

5: 674x1280 15.2ms
Speed: 71.1ms track, 15.2ms solution per image at shape (1, 3, 674, 1280)

6: 674x1280 17.5ms
Speed: 70.8ms track, 17.5ms solution per image at shape (1, 3, 674, 1280)

7: 674x1280 15.6ms
Speed: 70.7ms track, 15.6ms solution per image at shape (1, 3, 674, 1280)

8: 674x1280 15.4ms
Speed: 71.8ms track, 15.4ms solution per image at shape (1, 3, 674, 1280)

9: 674x1280 15.2ms
Speed: 73.0ms track, 15.2ms solution per image at shape (1, 3, 674, 1280)

10: 674x1280 15.1ms
Speed: 73.0ms track, 15.1ms solution pe

In [ ]:
cv2.destroyAllWindows()

: 

<img width="640" height="360" alt="queue-management-asset" src="https://github.com/ultralytics/notebooks/releases/download/v0.0.0/queue-management-img.png" />

Crafted with 💙 by [Ultralytics](https://ultralytics.com/)  

Explore and star the [Ultralytics Notebooks](https://github.com/ultralytics/notebooks/) to supercharge your AI journey! 🚀